In [14]:
import util
from journal_api.journal_api import JournalAPI
from IPython.display import JSON
import xml.etree.ElementTree as ET
import json
import numpy as np
import pandas as pd
import logging
import yaml
from journal_api import semanticscholar_api

In [15]:
with open('logging_config.yaml', 'r') as config_file:
    log_config = yaml.safe_load(config_file)
logging.config.dictConfig(log_config)

In [16]:
title = "Design and Control of Roller Grasper V2 for In-Hand Manipulation"

In [17]:
journal = JournalAPI()
dt = journal.search(title, 2018, find_references=False, fix_references=False)
JSON(dt)

2023-08-28 16:21:07,709 - DEBUG - ss.search: Found! title=Design and Control of Roller Grasper V2 for In-Hand Manipulation


<IPython.core.display.JSON object>

In [18]:
def source_target(source, target):
    ret = pd.DataFrame(columns=["source", "target"])
    for t in target:
        ret = ret.append({"source": source, "target": t}, ignore_index=True)
    return ret

In [31]:
df = None
def source_target_generator(title, title_list, level = 1, start_year = 2017, fix_references = False):
    global df
    ret = pd.DataFrame(columns=["source", "target"])
    if(level > 0):
        logging.info(f"source_target_generator: level={level}, len={len(title_list)}, title={title}")
        for index, title in enumerate(title_list):
            logging.info(f"source_target_generator: ({index+1}/{len(title_list)}) title={title}")
            src = journal.search(title, start_year, find_references=True, fix_references=fix_references)
            if src != None:
                # TODO: Create global variable to store detailed info of paper
                if df is None or df.empty:
                    df = pd.DataFrame([src])
                else:
                    matching_rows = df[(df["title"] == src["title"])]
                    if matching_rows.empty:
                        new_row = pd.Series(src)
                        df = df.append(new_row, ignore_index=True)
                    else:
                        logging.debug(f"source_target_generator: Title already exists. title={src['title']}")
                
                st = source_target(src["title"],src["references"])
                ret = pd.concat([ret, st], ignore_index=True)
                lv = level - 1
                if lv == 1:
                    ret = pd.concat([ret, source_target_generator(src["title"], src["references"], lv, start_year)], ignore_index=True)
                else:
                    ret = pd.concat([ret, source_target_generator(src["title"], src["references"], lv, start_year, fix_references)], ignore_index=True)
            
    return ret

In [32]:
x = source_target_generator("", [title], 2, 2018, False)

2023-08-28 16:34:19,819 - INFO - source_target_generator: level=2, len=1, title=
2023-08-28 16:34:19,821 - INFO - source_target_generator: (1/1) title=Design and Control of Roller Grasper V2 for In-Hand Manipulation
2023-08-28 16:34:20,429 - DEBUG - ss.search: Found! title=Design and Control of Roller Grasper V2 for In-Hand Manipulation
2023-08-28 16:34:20,448 - INFO - source_target_generator: level=1, len=8, title=Design and Control of Roller Grasper V2 for In-Hand Manipulation
2023-08-28 16:34:20,450 - INFO - source_target_generator: (1/8) title=Concept2Robot: Learning manipulation concepts from instructions and human demonstrations
2023-08-28 16:34:21,049 - DEBUG - ss.search: Found! title=Concept2Robot: Learning manipulation concepts from instructions and human demonstrations
2023-08-28 16:34:21,115 - INFO - source_target_generator: (2/8) title=Design of a Roller-Based Dexterous Hand for Object Grasping and Within-Hand Manipulation
2023-08-28 16:34:21,600 - DEBUG - ss.search: Found!

In [33]:
df

,paperId,title,abstract,year,fieldsOfStudy,tldr,authors,references,source
0,5d0c16d8532b4514dc4a330562e44d21c3dbaeff,Design and Control of Roller Grasper V2 for In...,The ability to perform in-hand manipulation st...,2020,"[Computer Science, Engineering]",This work presents a novel non-anthropomorphic...,"[Shenli Yuan, Lin Shao, Connor L. Yako, Alexan...",[Concept2Robot: Learning manipulation concepts...,semanticscholar
1,c328a1ea31eaa4b357e584285ffb6ff2671b5bd4,Concept2Robot: Learning manipulation concepts ...,We aim to endow a robot with the ability to le...,2020,[Computer Science],This work aims to endow a robot with the abili...,"[Lin Shao, Toki Migimatsu, Qiang Zhang, Karen ...",[Model-Based Inverse Reinforcement Learning fr...,semanticscholar
2,b019519f00715061b4fa05c7e8eda768f3b4779d,Design of a Roller-Based Dexterous Hand for Ob...,This paper describes the development of a nove...,2020,[Computer Science],This paper describes the development of a nove...,"[Shenli Yuan, A. D. Epps, Jerome B. Nowak, J. ...",[Design\nand Analysis of a Multimodal Grasper ...,semanticscholar
3,e5a3aca5321fc0e1b70fb51e4c4b6ff9dfbdb05d,Design\nand Analysis of a Multimodal Grasper H...,"\n This paper presents the design, analysis, a...",2019,[Computer Science],"This paper presents the design, analysis, and ...","[Nagamanikandan Govindan, Asokan Thondiyath]",[Variable-Friction Finger Surfaces to Enable W...,semanticscholar
4,f12e6e06816b1ff34bb0673a9059065458c43bc5,A Century of Robotic Hands,This article reports on the state of the art o...,2019,"[Computer Science, Engineering]",,"[C. Piazza, G. Grioli, M. G. Catalano, A. Bicchi]",[The evolution of functional hand replacement:...,semanticscholar
5,ec5cd7e50867ee305e6adb0592e11252a0977b91,Dual-Arm In-Hand Manipulation and Regrasping U...,This work focuses on the problem of in-hand ma...,2019,[Computer Science],This work proposes Dexterous Manipulation Grap...,"[S. Cruciani, Kaiyu Hang, Christian Smith, D. ...",[Preparatory Manipulation Planning Using Autom...,semanticscholar
6,d37a34c204a8beefcaef4dddddb7a90c16e973d4,Learning dexterous in-hand manipulation,We use reinforcement learning (RL) to learn de...,2018,"[Computer Science, Mathematics]",This work uses reinforcement learning (RL) to ...,"[Marcin Andrychowicz, Bowen Baker, Maciek Choc...",[The minimal exponent and $k$-rationality for ...,semanticscholar
7,9961da04571b16d282df74b624003ebb5f294063,In-Hand Manipulation via Motion Cones,"In this paper, we present the mechanics and al...",2018,"[Computer Science, Engineering]",It is shown that the motion cone is defined by...,"[Nikhil Chavan Dafle, Rachel Holladay, Alberto...","[Pushing revisited: Differential flatness, tra...",semanticscholar
8,6a00746fc98248da0db1e6074090c2d513f9487d,Dexterous Manipulation Graphs,We propose the Dexterous Manipulation Graph as...,2018,"[Computer Science, Engineering]",The Dexterous Manipulation Graph is proposed a...,"[S. Cruciani, Christian Smith, D. Kragic, Kaiy...",[Rotation (Fig. 2b)],semanticscholar


In [36]:
df.to_json("aaa.json", orient="records", indent=2)

In [ ]:
x

In [ ]:
x.to_csv("test_1.csv", index=False)

In [ ]:
import json

# Open the JSON file for reading
with open("output.json", "r") as json_file:
    data = json.load(json_file)

journal = JournalAPI()
for d in data["nodes"]:
    dt = journal.search(d["name"], 2018, find_references=False, fix_references=False)
    print(dt)

In [ ]:
from collections import defaultdict
from pyvis.network import Network

net = Network("1200px", "1200px", notebook=True)

# Example dummy data
sources = x["source"]
targets = x["target"]
sources_short = [source[:100] + "..." for source in sources]
targets_short = [target[:100] + "..." for target in targets]

# Create a dictionary to store the count of each target
target_counts = defaultdict(int)
for t in targets_short:
    target_counts[t] += 10
    if t == "...":
        target_counts[t] = 1

# Create unique node labels for each occurrence of a node
unique_nodes = {}
for node in set(sources_short + targets_short):
    count = target_counts[node]
    size = count if count > 1 else 1  # Ensuring the smallest size is 1
    unique_label = f"{node} ({count})"
    net.add_node(unique_label, size=size)
    unique_nodes[node] = unique_label

# Add edges to the graph using the unique node labels
for s, t in zip(sources_short, targets_short):
    net.add_edge(unique_nodes[s], unique_nodes[t])

# Step 3: Visualize the graph in Jupyter Notebook
net.show("network_graph2.html")


In [ ]:
ref_pd = util.string_to_df(xy["refs"][1])

In [ ]:
xy_refs = generate_title_auths_refs(ref_pd)
xy_refs

In [ ]:
xy_refs["title"]

In [ ]:
json_data = xy_refs["title"].to_json(orient='records')

# Specify the output file path
output_file = 'data.json'

# Write JSON data to a file
with open(output_file, 'w') as file:
    file.write(json_data)

In [ ]:
crossref = util.crossref_search(title)
json.dumps(crossref, ensure_ascii=False)

In [ ]:
cref = crossref_search_by_doi("10.1177/0278364914559753")

In [ ]:
crossref = crossref_similar_search(title,10)
crossref

In [ ]:
st = "ALVINN: An Autonomous Land Vehicle in a Neural Network"
tt = google_search_books(st)
tt

In [ ]:
arxiv_search("A reduction of imitation learning and structured prediction to no-regret online learning")

In [ ]:
import requests
import json

def crossref_keywords_search(keywords, rows):
    """
    crossref_search
    """
    # Specify the API endpoint
    ENDPOINT = "https://api.crossref.org/works"

    # Set up the query parameters
    params = {
        "query": " OR ".join(keywords),
        "rows": rows  # Number of results to retrieve
    }

    # Send the API request
    response = requests.get(ENDPOINT, params=params, timeout=10)

    # Parse the JSON response
    data = response.json()

    paper_list = []
    # Extract the relevant information from the response
    if data["status"] == "ok":
        papers = data["message"]["items"]
        if len(papers) > 0:
            for paper in papers:
                paper_list.append(paper)
    return paper_list

keywords = ["machine learning", "deep learning", "artificial intelligence"]
dois = crossref_keywords_search(keywords, 10)
print(json.dumps(dois))

In [ ]:
" OR ".join(keywords)

In [ ]:
import requests

def search_similar_papers(keywords):
    base_url = 'https://api.crossref.org/works'
    
    # Create the query string with the specified keywords
    query = ' '.join(f'title:{keyword} abstract:{keyword}' for keyword in keywords)
    
    # Prepare the request URL
    url = f'{base_url}?query={query}&rows=10'  # Limiting the response to 10 papers
    
    try:
        # Send the HTTP GET request
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            data = response.json()
            
            # Extract the relevant information from the response
            papers = data['message']['items']
            
            # Process the papers
            for paper in papers:
                title = paper['title'][0]
                authors = ""
                if paper.get("author") != None:
                    lst_authors = paper.get("author")
                    for author in lst_authors:
                        auth_name = author["given"] + " " + author["family"]
                        authors = authors + auth_name + ", "
                
                doi = paper['DOI']
                
                # Print the information
                print(f'Title: {title}')
                print(f'Authors: {authors}')
                print(f'DOI: {doi}')
                print('---')
        else:
            print(f'Request failed with status code {response.status_code}')
    except requests.exceptions.RequestException as e:
        print(f'An error occurred: {e}')

# Example usage
keywords = ['finger joint montions', 'spherical rolling fingertips', 'handcrafted control', 'sophisticated object manipulation', 'roller grasper']
search_similar_papers(keywords)

In [ ]:
t = [True, True, True, True]
t
if not any(t):
    print("True")

In [ ]:
for i in range(3):
    print(i)

In [ ]:
from pyvis.network import Network
import networkx as nx

# Create a sample network graph using NetworkX
G = nx.Graph()
G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 1)])

# Create a PyVis Network object
net = Network(notebook=True)

# Add nodes and edges to the PyVis Network
for node in G.nodes():
    net.add_node(node, label=f'Node {node}')

for edge in G.edges():
    net.add_edge(edge[0], edge[1])

# Define the click handler function
def show_text(node_id):
    node = net.nodes[node_id]
    if 'title' in node:
        del node['title']
    else:
        node['title'] = f'This is Node {node_id}'

# Bind the click handler to the 'on_click' event
net.add_event_listener("selectNode", show_text)

# Show the network graph
net.show("example.html")